# AI Job Market Data Cleaning 
This notebook performs comprehensive data cleaning on the AI job market dataset.

## 1. Import Libraries 

Import necessary Python libraries for data cleaning and analysis.


In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## 2. Load Dataset 

Load the AI job market dataset from CSV file.


In [2]:
# Load data
df = pd.read_csv('../dataset/ai_job_market.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

Dataset shape: (2000, 12)

Column names: ['job_id', 'company_name', 'industry', 'job_title', 'skills_required', 'experience_level', 'employment_type', 'location', 'salary_range_usd', 'posted_date', 'company_size', 'tools_preferred']

First few rows:


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,company_size,tools_preferred
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025/8/20,Large,"KDB+, LangChain"
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024/3/22,Large,"FastAPI, KDB+, TensorFlow"
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025/9/18,Large,"BigQuery, PyTorch, Scikit-learn"
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024/5/8,Large,"TensorFlow, BigQuery, MLflow"
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025/2/24,Large,"PyTorch, LangChain"


## 3. Check Missing Values 

Identify and analyze missing values in the dataset.

In [3]:
# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_data = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percentage
})

print("Missing Values Analysis:")
print(missing_data[missing_data['Missing Count'] > 0])
print(f"\nTotal missing values: {missing_values.sum()}")

Missing Values Analysis:
Empty DataFrame
Columns: [Missing Count, Percentage]
Index: []

Total missing values: 0


## 4. Handle Missing Values

Handle missing values using appropriate strategies for different columns.

In [4]:
# Create a copy for cleaning
df_cleaned = df.copy()

# Handle missing values in text columns with 'Unknown'
text_columns = ['company_name', 'industry', 'job_title', 'skills_required', 
                'tools_preferred', 'location']
for col in text_columns:
    if col in df_cleaned.columns:
        df_cleaned[col].fillna('Unknown', inplace=True)

# Handle missing values in categorical columns with mode
categorical_columns = ['experience_level', 'employment_type', 'company_size']
for col in categorical_columns:
    if col in df_cleaned.columns:
        mode_value = df_cleaned[col].mode()[0] if not df_cleaned[col].mode().empty else 'Unknown'
        df_cleaned[col].fillna(mode_value, inplace=True)

# Drop rows with missing salary information (critical for analysis)
if 'salary_range_usd' in df_cleaned.columns:
    df_cleaned = df_cleaned.dropna(subset=['salary_range_usd'])

print(f"Rows after handling missing values: {len(df_cleaned)}")
print(f"Remaining missing values: {df_cleaned.isnull().sum().sum()}")

Rows after handling missing values: 2000
Remaining missing values: 0


## 5. Standardize Job Titles

Standardize job titles by converting to lowercase and removing extra spaces.

In [5]:
# Standardize job titles
df_cleaned['job_title_standardized'] = df_cleaned['job_title'].str.strip().str.title()

# Create job title categories
def categorize_job_title(title):
    title_lower = str(title).lower()
    if 'data scientist' in title_lower or 'data science' in title_lower:
        return 'Data Scientist'
    elif 'data analyst' in title_lower:
        return 'Data Analyst'
    elif 'ml engineer' in title_lower or 'machine learning' in title_lower:
        return 'ML Engineer'
    elif 'nlp' in title_lower:
        return 'NLP Engineer'
    elif 'computer vision' in title_lower or 'cv engineer' in title_lower:
        return 'Computer Vision Engineer'
    elif 'ai researcher' in title_lower or 'research scientist' in title_lower:
        return 'AI Researcher'
    elif 'ai product manager' in title_lower or 'product manager' in title_lower:
        return 'AI Product Manager'
    elif 'quant' in title_lower:
        return 'Quant Researcher'
    else:
        return 'Other'

df_cleaned['job_category'] = df_cleaned['job_title'].apply(categorize_job_title)

print("Job Title Distribution:")
print(df_cleaned['job_category'].value_counts())

Job Title Distribution:
job_category
Data Analyst                271
NLP Engineer                265
AI Product Manager          258
Quant Researcher            251
ML Engineer                 250
Data Scientist              238
AI Researcher               237
Computer Vision Engineer    230
Name: count, dtype: int64


## 6. Standardize Skills

Extract and standardize skills from the skills_required and tools_preferred columns.

In [6]:
# Function to clean and standardize skills
def clean_skills(skill_string):
    if pd.isna(skill_string) or skill_string == 'Unknown':
        return []
    # Remove quotes and split by comma
    skills = [skill.strip() for skill in str(skill_string).split(',')]
    # Standardize common variations
    skill_map = {
        'python': 'Python',
        'r': 'R',
        'sql': 'SQL',
        'pytorch': 'PyTorch',
        'tensorflow': 'TensorFlow',
        'keras': 'Keras',
        'scikit-learn': 'Scikit-learn',
        'pandas': 'Pandas',
        'numpy': 'NumPy',
        'excel': 'Excel',
        'aws': 'AWS',
        'azure': 'Azure',
        'gcp': 'GCP',
        'cuda': 'CUDA',
        'c++': 'C++',
        'flask': 'Flask',
        'fastapi': 'FastAPI',
        'mlflow': 'MLflow',
        'langchain': 'LangChain',
        'hugging face': 'Hugging Face',
        'power bi': 'Power BI',
        'reinforcement learning': 'Reinforcement Learning'
    }
    
    standardized_skills = []
    for skill in skills:
        skill_lower = skill.lower().strip()
        standardized_skills.append(skill_map.get(skill_lower, skill.strip()))
    
    return standardized_skills

# Apply standardization
df_cleaned['skills_list'] = df_cleaned['skills_required'].apply(clean_skills)
df_cleaned['tools_list'] = df_cleaned['tools_preferred'].apply(clean_skills)

# Count number of skills
df_cleaned['num_skills_required'] = df_cleaned['skills_list'].apply(len)
df_cleaned['num_tools_preferred'] = df_cleaned['tools_list'].apply(len)

print(f"Average number of skills required: {df_cleaned['num_skills_required'].mean():.2f}")
print(f"Average number of tools preferred: {df_cleaned['num_tools_preferred'].mean():.2f}")

Average number of skills required: 4.49
Average number of tools preferred: 1.98


## 7. Calculate Average Salary

Parse salary range and calculate average salary for analysis.

In [7]:
# Function to parse salary range and calculate average
def parse_salary(salary_string):
    if pd.isna(salary_string):
        return None, None, None
    
    # Extract numbers from salary range (format: "min-max")
    match = re.findall(r'(\d+)', str(salary_string))
    
    if len(match) >= 2:
        min_salary = int(match[0])
        max_salary = int(match[1])
        avg_salary = (min_salary + max_salary) / 2
        return min_salary, max_salary, avg_salary
    else:
        return None, None, None

# Apply salary parsing
salary_data = df_cleaned['salary_range_usd'].apply(parse_salary)
df_cleaned['salary_min'] = salary_data.apply(lambda x: x[0])
df_cleaned['salary_max'] = salary_data.apply(lambda x: x[1])
df_cleaned['salary_avg'] = salary_data.apply(lambda x: x[2])

# Create salary categories
def categorize_salary(avg_salary):
    if pd.isna(avg_salary):
        return 'Unknown'
    elif avg_salary < 70000:
        return 'Low (<70K)'
    elif avg_salary < 100000:
        return 'Mid (70K-100K)'
    elif avg_salary < 150000:
        return 'High (100K-150K)'
    else:
        return 'Very High (>150K)'

df_cleaned['salary_category'] = df_cleaned['salary_avg'].apply(categorize_salary)

print("Salary Statistics:")
print(f"Average Salary: ${df_cleaned['salary_avg'].mean():,.2f}")
print(f"Median Salary: ${df_cleaned['salary_avg'].median():,.2f}")
print(f"Min Salary: ${df_cleaned['salary_avg'].min():,.2f}")
print(f"Max Salary: ${df_cleaned['salary_avg'].max():,.2f}")
print(f"\nSalary Distribution:")
print(df_cleaned['salary_category'].value_counts())

Salary Statistics:
Average Salary: $123,040.00
Median Salary: $123,202.75
Min Salary: $47,578.50
Max Salary: $197,776.50

Salary Distribution:
salary_category
High (100K-150K)     900
Very High (>150K)    521
Mid (70K-100K)       448
Low (<70K)           131
Name: count, dtype: int64


## 8. Standardize Location Data

Parse and standardize location information into city, state/country codes, and regions.

In [8]:
# Function to parse location
def parse_location(location_string):
    if pd.isna(location_string) or location_string == 'Unknown':
        return None, None
    
    # Split by comma to get city and country code
    parts = str(location_string).split(',')
    if len(parts) >= 2:
        city = parts[0].strip()
        country_code = parts[1].strip()
        return city, country_code
    else:
        return location_string, None

# Apply location parsing
location_data = df_cleaned['location'].apply(parse_location)
df_cleaned['city'] = location_data.apply(lambda x: x[0])
df_cleaned['country_code'] = location_data.apply(lambda x: x[1])

# Map country codes to country names (ISO 3166-1 alpha-2)
country_code_to_name = {
    # North America
    'US': 'United States', 'CA': 'Canada', 'MX': 'Mexico',
    
    # Europe
    'GB': 'United Kingdom', 'DE': 'Germany', 'FR': 'France', 'IT': 'Italy',
    'ES': 'Spain', 'NL': 'Netherlands', 'SE': 'Sweden', 'NO': 'Norway',
    'PL': 'Poland', 'TR': 'Turkey', 'PT': 'Portugal', 'BE': 'Belgium',
    'RO': 'Romania', 'BG': 'Bulgaria', 'CZ': 'Czech Republic', 'HU': 'Hungary',
    'LT': 'Lithuania', 'LV': 'Latvia', 'LU': 'Luxembourg', 'AD': 'Andorra',
    'AG': 'Antigua and Barbuda', 'LR': 'Liberia', 'LC': 'Saint Lucia', 
    'BB': 'Barbados', 'BT': 'Bhutan', 'HR': 'Croatia', 'ME': 'Montenegro',
    'MC': 'Monaco', 'RS': 'Serbia', 'SC': 'Seychelles', 'AM': 'Armenia',
    'UA': 'Ukraine', 'KZ': 'Kazakhstan', 'GE': 'Georgia', 'FI': 'Finland',
    'BR': 'Brazil', 'AR': 'Argentina', 'NE': 'Niger', 'ID': 'Indonesia',
    'MA': 'Morocco', 'AZ': 'Azerbaijan',
    
    # Asia
    'CN': 'China', 'JP': 'Japan', 'IN': 'India', 'KR': 'South Korea',
    'SG': 'Singapore', 'TH': 'Thailand', 'MY': 'Malaysia', 'VN': 'Vietnam',
    'PH': 'Philippines', 'TW': 'Taiwan', 'HK': 'Hong Kong', 'PK': 'Pakistan',
    'BD': 'Bangladesh', 'IQ': 'Iraq', 'KH': 'Cambodia', 'MM': 'Myanmar',
    'KG': 'Kyrgyzstan', 'YE': 'Yemen', 'UZ': 'Uzbekistan', 'KW': 'Kuwait',
    'IR': 'Iran', 'KE': 'Kenya', 'NG': 'Nigeria',
    
    # Oceania
    'AU': 'Australia', 'NZ': 'New Zealand', 'FJ': 'Fiji', 'PG': 'Papua New Guinea',
    'KI': 'Kiribati', 'PW': 'Palau',
    
    # South America
    'CL': 'Chile', 'CO': 'Colombia', 'PE': 'Peru', 'VE': 'Venezuela',
    'UY': 'Uruguay', 'PY': 'Paraguay', 'GY': 'Guyana',
    
    # Africa
    'ZA': 'South Africa', 'EG': 'Egypt', 'ET': 'Ethiopia', 'GH': 'Ghana',
    'TZ': 'Tanzania', 'UG': 'Uganda', 'TD': 'Chad', 'GQ': 'Equatorial Guinea',
    'BF': 'Burkina Faso', 'SN': 'Senegal', 'TG': 'Togo', 'CI': "Côte d'Ivoire",
    'GW': 'Guinea-Bissau', 'BW': 'Botswana', 'LS': 'Lesotho', 'BS': 'Bahamas',
    'DO': 'Dominican Republic', 'MR': 'Mauritania', 'DZ': 'Algeria',
    'CF': 'Central African Republic', 'AO': 'Angola', 'SA': 'Saudi Arabia',
    
    # Central America / Caribbean
    'GT': 'Guatemala', 'HN': 'Honduras', 'SV': 'El Salvador', 'CR': 'Costa Rica',
    'PA': 'Panama', 'BZ': 'Belize', 'JM': 'Jamaica', 'TT': 'Trinidad and Tobago',
    'BN': 'Brunei', 'KN': 'Saint Kitts and Nevis', 'KM': 'Comoros',
    'SB': 'Solomon Islands', 'VC': 'Saint Vincent and the Grenadines',
    'QA': 'Qatar',
    
    # Other regions
    'RU': 'Russia', 'CU': 'Cuba', 'CM': 'Cameroon', 'BA': 'Bosnia and Herzegovina',
    'FM': 'Micronesia', 'LI': 'Liechtenstein', 'VU': 'Vanuatu',
    'PS': 'Palestine', 'MD': 'Moldova', 'ZW': 'Zimbabwe', 'MK': 'North Macedonia',
    'DM': 'Dominica', 'MG': 'Madagascar', 'MW': 'Malawi', 'BJ': 'Benin',
    'TL': 'Timor-Leste', 'BI': 'Burundi', 'SL': 'Sierra Leone'
}

# Map country codes to regions
country_to_region = {
    # North America
    'US': 'North America', 'CA': 'North America', 'MX': 'North America',
    'AR': 'South America', 'NE': 'Africa', 'ID': 'Asia',
    'MA': 'Africa', 'AZ': 'Asia',
    
    # Europe
    'GB': 'Europe', 'DE': 'Europe', 'FR': 'Europe', 'IT': 'Europe', 
    'ES': 'Europe', 'NL': 'Europe', 'SE': 'Europe', 'NO': 'Europe',
    'PL': 'Europe', 'TR': 'Europe', 'PT': 'Europe', 'BE': 'Europe',
    'RO': 'Europe', 'BG': 'Europe', 'CZ': 'Europe', 'HU': 'Europe',
    'LT': 'Europe', 'LV': 'Europe', 'LU': 'Europe', 'AD': 'Europe',
    'AG': 'Caribbean', 'LR': 'Africa', 'LC': 'Caribbean', 'BB': 'Caribbean',
    'BT': 'Asia', 'HR': 'Europe', 'ME': 'Europe', 'MC': 'Europe',
    'RS': 'Europe', 'SC': 'Africa', 'AM': 'Asia', 'UA': 'Europe',
    'KZ': 'Asia', 'GE': 'Asia', 'FI': 'Europe', 'BR': 'South America',
    
    # Asia
    'CN': 'Asia', 'JP': 'Asia', 'IN': 'Asia', 'KR': 'Asia', 'SG': 'Asia',
    'TH': 'Asia', 'MY': 'Asia', 'VN': 'Asia', 'PH': 'Asia', 'TW': 'Asia',
    'HK': 'Asia', 'PK': 'Asia', 'BD': 'Asia', 'IQ': 'Asia', 'KH': 'Asia',
    'MM': 'Asia', 'KG': 'Asia', 'YE': 'Asia', 'UZ': 'Asia', 'KW': 'Asia',
    'IR': 'Asia', 'KE': 'Africa', 'NG': 'Africa',
    
    # Oceania
    'AU': 'Oceania', 'NZ': 'Oceania', 'FJ': 'Oceania', 'PG': 'Oceania',
    'KI': 'Oceania', 'PW': 'Oceania',
    
    # South America
    'CL': 'South America', 'CO': 'South America',
    'PE': 'South America', 'VE': 'South America', 'UY': 'South America',
    'PY': 'South America', 'GY': 'South America',
    
    # Africa
    'ZA': 'Africa', 'EG': 'Africa', 'ET': 'Africa', 'GH': 'Africa',
    'TZ': 'Africa', 'UG': 'Africa', 'TD': 'Africa', 'GQ': 'Africa',
    'BF': 'Africa', 'SN': 'Africa', 'TG': 'Africa', 'CI': 'Africa',
    'GW': 'Africa', 'BW': 'Africa', 'LS': 'Africa', 'BS': 'Caribbean',
    'DO': 'Caribbean', 'MR': 'Africa', 'DZ': 'Africa',
    'CF': 'Africa', 'AO': 'Africa', 'SA': 'Middle East',
    
    # Central America / Caribbean
    'GT': 'Central America', 'HN': 'Central America', 'SV': 'Central America',
    'CR': 'Central America', 'PA': 'Central America', 'BZ': 'Central America',
    'JM': 'Caribbean', 'TT': 'Caribbean', 'BN': 'Asia', 'KN': 'Caribbean',
    'KM': 'Africa', 'SB': 'Oceania', 'VC': 'Caribbean', 'QA': 'Middle East',
    
    # Other regions
    'RU': 'Europe', 'CU': 'Caribbean', 'CM': 'Africa', 'BA': 'Europe',
    'FM': 'Oceania', 'LI': 'Europe', 'VU': 'Oceania',
    'PS': 'Middle East', 'MD': 'Europe', 'ZW': 'Africa', 'MK': 'Europe',
    'DM': 'Caribbean', 'MG': 'Africa', 'MW': 'Africa', 'BJ': 'Africa',
    'TL': 'Asia', 'BI': 'Africa', 'SL': 'Africa'
}

# Map country codes to country names and regions
df_cleaned['country'] = df_cleaned['country_code'].map(country_code_to_name)
df_cleaned['country'].fillna('Unknown', inplace=True)

df_cleaned['region'] = df_cleaned['country_code'].map(country_to_region)
df_cleaned['region'].fillna('Other', inplace=True)

print("Region Distribution:")
print(df_cleaned['region'].value_counts())
print(f"\nTotal unique cities: {df_cleaned['city'].nunique()}")
print(f"Total unique countries: {df_cleaned['country'].nunique()}")
print(f"\nSample countries:")
print(df_cleaned['country'].value_counts().head(10))

Region Distribution:
region
Other              591
Africa             336
Europe             332
Asia               280
Caribbean          136
Oceania            115
South America       84
Central America     62
North America       32
Middle East         32
Name: count, dtype: int64

Total unique cities: 1858
Total unique countries: 134

Sample countries:
country
Unknown              591
Papua New Guinea      19
Bhutan                18
Fiji                  18
Croatia               18
Barbados              18
Iraq                  17
Uzbekistan            16
Jamaica               16
Equatorial Guinea     16
Name: count, dtype: int64


## 9. Standardize and Categorize Industries

Standardize industry names and group them into broader categories.

In [9]:
# Standardize industry names
df_cleaned['industry_standardized'] = df_cleaned['industry'].str.strip().str.title()

# Function to categorize industries into broader groups
def categorize_industry(industry):
    if pd.isna(industry) or industry == 'Unknown':
        return 'Other'
    
    industry_lower = str(industry).lower()
    
    # Finance related
    if any(keyword in industry_lower for keyword in ['finance', 'banking', 'investment', 'insurance', 'fintech']):
        return 'Finance & Banking'
    
    # Technology
    elif any(keyword in industry_lower for keyword in ['tech', 'technology', 'software', 'it', 'computer']):
        return 'Technology'
    
    # Healthcare
    elif any(keyword in industry_lower for keyword in ['health', 'healthcare', 'medical', 'pharma', 'hospital', 'biotech']):
        return 'Healthcare'
    
    # Retail & E-commerce
    elif any(keyword in industry_lower for keyword in ['retail', 'e-commerce', 'ecommerce', 'shopping', 'commerce']):
        return 'Retail & E-commerce'
    
    # Education
    elif any(keyword in industry_lower for keyword in ['education', 'university', 'school', 'learning', 'edtech']):
        return 'Education'
    
    # Automotive & Transportation
    elif any(keyword in industry_lower for keyword in ['automotive', 'automobile', 'transport', 'vehicle', 'car']):
        return 'Automotive & Transportation'
    
    # Manufacturing
    elif any(keyword in industry_lower for keyword in ['manufacturing', 'industrial', 'production']):
        return 'Manufacturing'
    
    # Media & Entertainment
    elif any(keyword in industry_lower for keyword in ['media', 'entertainment', 'gaming', 'streaming']):
        return 'Media & Entertainment'
    
    # Energy & Utilities
    elif any(keyword in industry_lower for keyword in ['energy', 'utility', 'utilities', 'power', 'oil', 'gas']):
        return 'Energy & Utilities'
    
    # Telecommunications
    elif any(keyword in industry_lower for keyword in ['telecom', 'telecommunications', 'communication']):
        return 'Telecommunications'
    
    # Consulting
    elif any(keyword in industry_lower for keyword in ['consulting', 'advisory', 'professional services']):
        return 'Consulting'
    
    else:
        return 'Other'

df_cleaned['industry_category'] = df_cleaned['industry'].apply(categorize_industry)

print("Original Industries:")
print(df_cleaned['industry_standardized'].value_counts())
print("\n" + "="*60 + "\n")
print("Industry Categories:")
print(df_cleaned['industry_category'].value_counts())

Original Industries:
industry_standardized
Automotive    300
Education     294
Retail        293
E-Commerce    291
Finance       279
Tech          274
Healthcare    269
Name: count, dtype: int64


Industry Categories:
industry_category
Retail & E-commerce            584
Automotive & Transportation    300
Education                      294
Finance & Banking              279
Technology                     274
Healthcare                     269
Name: count, dtype: int64


## 10. Standardize Experience Level and Employment Type

Standardize experience level and employment type for consistency.

In [10]:
# Standardize experience level
experience_mapping = {
    'entry': 'Entry',
    'junior': 'Entry',
    'mid': 'Mid',
    'intermediate': 'Mid',
    'senior': 'Senior',
    'lead': 'Senior',
    'principal': 'Senior',
    'expert': 'Senior'
}

df_cleaned['experience_level_standardized'] = df_cleaned['experience_level'].str.lower().map(
    experience_mapping
).fillna(df_cleaned['experience_level'])

# Standardize employment type
employment_mapping = {
    'full-time': 'Full-Time',
    'fulltime': 'Full-Time',
    'full time': 'Full-Time',
    'part-time': 'Part-Time',
    'parttime': 'Part-Time',
    'part time': 'Part-Time',
    'contract': 'Contract',
    'contractor': 'Contract',
    'freelance': 'Contract',
    'internship': 'Internship',
    'intern': 'Internship',
    'remote': 'Remote',
    'temporary': 'Temporary',
    'temp': 'Temporary'
}

df_cleaned['employment_type_standardized'] = df_cleaned['employment_type'].str.lower().map(
    employment_mapping
).fillna(df_cleaned['employment_type'])

print("Experience Level Distribution:")
print(df_cleaned['experience_level_standardized'].value_counts())
print("\n" + "="*60 + "\n")
print("Employment Type Distribution:")
print(df_cleaned['employment_type_standardized'].value_counts())

Experience Level Distribution:
experience_level_standardized
Entry     702
Mid       668
Senior    630
Name: count, dtype: int64


Employment Type Distribution:
employment_type_standardized
Internship    574
Full-Time     509
Contract      465
Remote        452
Name: count, dtype: int64


## 11. Standardize Company Size

Standardize company size categories for consistency.

In [11]:
# Standardize company size
company_size_mapping = {
    'startup': 'Startup',
    'small': 'Small',
    'mid': 'Mid',
    'medium': 'Mid',
    'mid-size': 'Mid',
    'large': 'Large',
    'enterprise': 'Large'
}

df_cleaned['company_size_standardized'] = df_cleaned['company_size'].str.lower().map(
    company_size_mapping
).fillna(df_cleaned['company_size'])

print("Company Size Distribution:")
print(df_cleaned['company_size_standardized'].value_counts())

Company Size Distribution:
company_size_standardized
Startup    672
Mid        671
Large      657
Name: count, dtype: int64


## 12. Parse and Standardize Posted Date

Convert posted date to datetime format and extract useful time features.

In [12]:
# Convert posted_date to datetime
df_cleaned['posted_date_parsed'] = pd.to_datetime(df_cleaned['posted_date'], format='%Y/%m/%d', errors='coerce')

# Extract time-based features
df_cleaned['posted_year'] = df_cleaned['posted_date_parsed'].dt.year
df_cleaned['posted_month'] = df_cleaned['posted_date_parsed'].dt.month
df_cleaned['posted_quarter'] = df_cleaned['posted_date_parsed'].dt.quarter
df_cleaned['posted_month_name'] = df_cleaned['posted_date_parsed'].dt.strftime('%B')

print("Date Range:")
print(f"Earliest posting: {df_cleaned['posted_date_parsed'].min()}")
print(f"Latest posting: {df_cleaned['posted_date_parsed'].max()}")
print(f"\nPostings by Year:")
print(df_cleaned['posted_year'].value_counts().sort_index())
print(f"\nPostings by Quarter:")
print(df_cleaned['posted_quarter'].value_counts().sort_index())

Date Range:
Earliest posting: 2023-09-21 00:00:00
Latest posting: 2025-09-19 00:00:00

Postings by Year:
posted_year
2023     271
2024    1006
2025     723
Name: count, dtype: int64

Postings by Quarter:
posted_quarter
1    529
2    447
3    521
4    503
Name: count, dtype: int64


## 13. Data Quality Summary

Summarize data quality after cleaning operations.

In [13]:
# Data quality summary
print("="*70)
print("DATA CLEANING SUMMARY")
print("="*70)

print(f"\n1. Dataset Size:")
print(f"   Original rows: {len(df)}")
print(f"   Cleaned rows: {len(df_cleaned)}")
print(f"   Rows removed: {len(df) - len(df_cleaned)}")

print(f"\n2. Missing Values:")
print(f"   Total missing values: {df_cleaned.isnull().sum().sum()}")

print(f"\n3. New Features Created:")
new_features = [
    'job_title_standardized', 'job_category', 'skills_list', 'tools_list',
    'num_skills_required', 'num_tools_preferred', 'salary_min', 'salary_max',
    'salary_avg', 'salary_category', 'city', 'country_code', 'country', 'region',
    'industry_standardized', 'industry_category', 'experience_level_standardized',
    'employment_type_standardized', 'company_size_standardized', 'posted_date_parsed',
    'posted_year', 'posted_month', 'posted_quarter', 'posted_month_name'
]
print(f"   Total new features: {len(new_features)}")
print(f"   Feature names: {', '.join(new_features[:10])}...")

print(f"\n4. Data Standardization:")
print(f"   ✓ Job titles standardized and categorized")
print(f"   ✓ Skills and tools parsed and standardized")
print(f"   ✓ Salary ranges converted to average values")
print(f"   ✓ Locations parsed into city, country code, country name, and region")
print(f"   ✓ Industries categorized into broader groups")
print(f"   ✓ Experience levels standardized")
print(f"   ✓ Employment types standardized")
print(f"   ✓ Company sizes standardized")
print(f"   ✓ Dates parsed and time features extracted")

print(f"\n5. Data Types:")
print(df_cleaned.dtypes)

print("\n" + "="*70)

DATA CLEANING SUMMARY

1. Dataset Size:
   Original rows: 2000
   Cleaned rows: 2000
   Rows removed: 0

2. Missing Values:
   Total missing values: 0

3. New Features Created:
   Total new features: 24
   Feature names: job_title_standardized, job_category, skills_list, tools_list, num_skills_required, num_tools_preferred, salary_min, salary_max, salary_avg, salary_category...

4. Data Standardization:
   ✓ Job titles standardized and categorized
   ✓ Skills and tools parsed and standardized
   ✓ Salary ranges converted to average values
   ✓ Locations parsed into city, country code, country name, and region
   ✓ Industries categorized into broader groups
   ✓ Experience levels standardized
   ✓ Employment types standardized
   ✓ Company sizes standardized
   ✓ Dates parsed and time features extracted

5. Data Types:
job_id                                    int64
company_name                             object
industry                                 object
job_title                 

## 14. View Sample of Cleaned Data

Display a sample of the cleaned dataset with key features.

In [14]:
# Display key columns from cleaned data
key_columns = [
    'job_id', 'company_name', 'job_category', 'industry_category',
    'experience_level_standardized', 'employment_type_standardized',
    'salary_avg', 'city', 'region', 'num_skills_required'
]

print("Sample of Cleaned Data (Key Features):")
print("="*100)
df_cleaned[key_columns].head(10)

Sample of Cleaned Data (Key Features):


,job_id,company_name,job_category,industry_category,experience_level_standardized,employment_type_standardized,salary_avg,city,region,num_skills_required
0,1,Foster and Sons,Data Analyst,Healthcare,Mid,Full-Time,101229.0,Tracybury,South America,6
1,2,"Boyd, Myers and Ramirez",Computer Vision Engineer,Technology,Senior,Full-Time,111699.0,Lake Scott,Caribbean,4
2,3,King Inc,Quant Researcher,Technology,Entry,Full-Time,170850.0,East Paige,Africa,6
3,4,"Cooper, Archer and Lynch",AI Product Manager,Technology,Mid,Full-Time,87325.5,Perezview,Europe,6
4,5,Hall LLC,Data Scientist,Finance & Banking,Senior,Contract,117053.5,North Desireeland,Africa,4
5,6,Ellis PLC,AI Product Manager,Retail & E-commerce,Senior,Remote,136675.0,South Kevin,Africa,4
6,7,Matthews-Moses,Data Analyst,Automotive & Transportation,Mid,Full-Time,79584.5,West Shawn,Africa,4
7,8,Mullins Ltd,Data Scientist,Education,Entry,Internship,72588.0,Port Hailey,Europe,3
8,9,Aguilar PLC,ML Engineer,Healthcare,Entry,Internship,161371.5,Butlermouth,Europe,4
9,10,Parks LLC,Computer Vision Engineer,Automotive & Transportation,Senior,Full-Time,122195.0,Nicoleshire,Europe,3


## 15. Export Cleaned Data

Save the cleaned dataset to a new CSV file for further analysis.

In [15]:
# Export cleaned data to CSV
output_path = '../dataset/ai_job_market_cleaned.csv'
df_cleaned.to_csv(output_path, index=False)

print(f"✓ Cleaned data exported successfully to: {output_path}")
print(f"✓ Total records: {len(df_cleaned)}")
print(f"✓ Total columns: {len(df_cleaned.columns)}")
print(f"\nCleaned dataset columns:")
print(df_cleaned.columns.tolist())

✓ Cleaned data exported successfully to: ../dataset/ai_job_market_cleaned.csv
✓ Total records: 2000
✓ Total columns: 36

Cleaned dataset columns:
['job_id', 'company_name', 'industry', 'job_title', 'skills_required', 'experience_level', 'employment_type', 'location', 'salary_range_usd', 'posted_date', 'company_size', 'tools_preferred', 'job_title_standardized', 'job_category', 'skills_list', 'tools_list', 'num_skills_required', 'num_tools_preferred', 'salary_min', 'salary_max', 'salary_avg', 'salary_category', 'city', 'country_code', 'country', 'region', 'industry_standardized', 'industry_category', 'experience_level_standardized', 'employment_type_standardized', 'company_size_standardized', 'posted_date_parsed', 'posted_year', 'posted_month', 'posted_quarter', 'posted_month_name']


## Summary

### Data Cleaning Operations Performed:

1. **Missing Value Handling**
   - Filled text columns with 'Unknown'
   - Filled categorical columns with mode values
   - Dropped rows with missing salary information

2. **Text Standardization**
   - Standardized job titles and created job categories
   - Standardized and categorized industries
   - Standardized experience levels and employment types
   - Standardized company sizes

3. **Skills Processing**
   - Extracted and standardized skills from comma-separated lists
   - Created skill count features
   - Applied consistent naming conventions

4. **Salary Processing**
   - Parsed salary ranges into min, max, and average values
   - Created salary categories for grouping
   - Enabled numerical analysis of compensation

5. **Location Processing**
   - Parsed locations into city and country code
   - Mapped country codes to full country names
   - Mapped country codes to geographic regions
   - Created hierarchical location structure (City → Country Code → Country Name → Region)

6. **Date Processing**
   - Converted dates to datetime format
   - Extracted time-based features (year, month, quarter)

### Key Features Created:

- `job_category`: Categorized job titles
- `industry_category`: Broader industry groupings
- `salary_avg`: Average salary for analysis
- `country`: Full country names from country codes
- `region`: Geographic regions for location analysis
- `num_skills_required`: Count of required skills
- `posted_year`, `posted_month`, `posted_quarter`: Time-based features

### Data Ready For:

- Exploratory Data Analysis (EDA)
- Statistical Analysis
- Machine Learning Models
- Visualization and Reporting